__import__ required Python libraries

In [1]:
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import requests

__generate__ the hockey-reference url

In [2]:
stats_month = 2
stats_day = 9
stats_year = 2018
stats_webpage = 'https://www.hockey-reference.com/friv/dailyleaders.fcgi?month={}&day={}&year={}'.format(
                 stats_month, stats_day, stats_year)

__create__ the soup object

In [3]:
r = requests.get(stats_webpage)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

__get__ all records from the goalies table

In [4]:
goalies_table = soup.find(id='goalies').find_all('td')
goalies_tbody = [cell.get_text() for cell in goalies_table]
goalies_records = [tuple(goalies_tbody[row: row + 15]) for row in range(0, len(goalies_tbody), 15)]

__make__ a list of player ids

In [5]:
player_id_list = []

for player_id in range(len(goalies_table)):
    try:
        player_id_list.append(goalies_table[player_id]['data-append-csv'])
    except:
        pass

__load__ the goalies data into a Pandas DataFrame

In [6]:
columns_list = ['Player', 'Position','Team', 'Home Away', 'Opponent', 'Result', 'Boxscore', 'DEC', 'GA', 'SA', 'SV', 'SV%', 'SO', 'PIM', 'TOI']
df_goalies = pd.DataFrame.from_records(goalies_records, columns=columns_list)
df_goalies.insert(0, 'Date', datetime.datetime(stats_year, stats_month, stats_day))

__concatenate__ the Player IDs to the goalies DataFrame

In [7]:
df_goalies_id = pd.DataFrame(player_id_list, columns=['Player ID'])
df_concat = pd.concat([df_goalies_id, df_goalies], axis=1)

__delete__ the *Boxscore* column and __replace__ the contents of the *Home Away* column

In [8]:
del df_concat['Boxscore']
df_concat['Home Away'] = df_concat['Home Away'].apply(lambda x: 'Home' if x == '' else 'Away') 

__index__ the goalies DataFrame by *Date* and *Player ID* and __sort__ by the index

In [9]:
df_concat.set_index(['Date','Player ID'],inplace=True, drop=True)
df_concat.sort_index(axis=0, inplace=True, ascending=True)

__result__ of the above code

In [10]:
print(df_concat)

                                 Player Position Team Home Away Opponent  \
Date       Player ID                                                       
2018-02-09 allenja01         Jake Allen        G  STL      Away      WPG   
           bobrose01   Sergei Bobrovsky        G  CBJ      Away      WSH   
           darlisc01      Scott Darling        G  CAR      Home      VAN   
           gibsojo02        John Gibson        G  ANA      Home      EDM   
           greisth01      Thomas Greiss        G  NYI      Home      DET   
           halakja01     Jaroslav Halak        G  NYI      Home      DET   
           helleco01  Connor Hellebuyck        G  WPG      Home      STL   
           holtbbr01      Braden Holtby        G  WSH      Home      CBJ   
           lehtoka01      Kari Lehtonen        G  DAL      Home      PIT   
           lundqhe01   Henrik Lundqvist        G  NYR      Home      CGY   
           marksja02    Jacob Markstrom        G  VAN      Away      CAR   
           m